In [1]:
import numpy as np
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import threading
import concurrent.futures

In [2]:
# Customize Chrome browser settings
chrome_options = Options()                                     
chrome_options.add_argument('--no-sandbox')                     # Them tuy chon de chay Chrome ko co sandbox, thuong duoc su dung de tranh cac van de ve quyen truy cap he thong
chrome_options.add_argument('--disable-notifications')          # Tat thong bao tu trinh duyet
chrome_options.add_argument('--disable-infobars')               # Tat thanh thong tin cua trinh duyet
#chrome_options.add_argument('--headless')

In [4]:
def getDataIn1Page(search_, page_, data_lock, data_crawled):
    base_link = 'https://www.lazada.vn/catalog/?page={page}&q={search}'
    link = base_link.format(page = page_, search = search_)
    # Access
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.maximize_window()
    driver.get(link)
    driver.implicitly_wait(2)

    # Get link/title
    elems = driver.find_elements(By.CSS_SELECTOR , ".RfADt [href]")
    titles = [elem.text for elem in elems]
    links = [elem.get_attribute('href') for elem in elems]

    # Get type
    type_name = [elem.get_attribute("value") for elem in driver.find_elements(By.CSS_SELECTOR, ".search-box__input--O34g")]
    type_name = type_name * len(titles)

    # Get price sale
    elems_price_sale = driver.find_elements(By.CSS_SELECTOR , ".aBrP0")
    prices = [elem_price.text for elem_price in elems_price_sale]

    # Get discount, sold, preview
    discount_percent_list, sold_list, preview_list = [], [], []
    for i in range(1, len(titles)+1):
        try:
            discount_percent = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[4]/span".format(i))
            discount_percent_list.append(discount_percent.text)
        except NoSuchElementException:
            discount_percent_list.append(None)

        try:
            sold = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/span[1]/span[1]".format(i))
            sold_list.append(sold.text)
        except NoSuchElementException:
            sold_list.append(None)

        try:
            preview = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/div/span".format(i))
            preview_list.append(preview.text)
        except NoSuchElementException:
            preview_list.append(None)
    
    # Get Location
    elems_locations = driver.find_elements(By.CSS_SELECTOR , ".oa6ri ")
    locations = [elem.text for elem in elems_locations]

    # Cập nhật dữ liệu vào biến chung
    with data_lock:
        data_crawled['type'].extend(type_name)
        data_crawled['title'].extend(titles)
        data_crawled['links'].extend(links)
        data_crawled['price'].extend(prices)
        data_crawled['discount_percent'].extend(discount_percent_list)
        data_crawled['sold'].extend(sold_list)
        data_crawled['preview'].extend(preview_list)
        data_crawled['location'].extend(locations)
    
    driver.quit()

In [5]:
dataProducts = {
    "type": [],
    "title": [],
    "links": [],
    "price": [],
    "discount_percent": [],
    "sold": [],
    "preview": [],
    "location": []
}

In [6]:
def crawlDataWithMultiplesThread(product, num_pages, num_threads_per_loop):
    # Lock để đảm bảo thread-safe khi cập nhật dữ liệu
    data_lock = threading.Lock()
    
    # Lặp qua các trang và chạy các luồng
    for start_page in tqdm(range(1, num_pages + 1, num_threads_per_loop), desc="Crawling Data"):
        threads = []
        for page in range(start_page, start_page + num_threads_per_loop):
            thread = threading.Thread(target=getDataIn1Page, args=(product, page, data_lock, dataProducts))
            threads.append(thread)
            thread.start()

        # Đợi tất cả các luồng hoàn thành và thu thập dữ liệu đã crawl
        for thread in threads:
            thread.join()

        print(f"Crawling completed from pages {start_page} to {min(start_page + num_threads_per_loop - 1, num_pages)}")
        sleep(2)

    print("Crawling completed.")

    return dataProducts

In [7]:
dataProducts = crawlDataWithMultiplesThread("quần+áo", 1, 1)

Crawling Data:   0%|          | 0/1 [00:00<?, ?it/s]

Crawling Data: 100%|██████████| 1/1 [00:58<00:00, 58.42s/it]

Crawling completed from pages 1 to 1
Crawling completed.


In [8]:
dataProducts

{'type': ['quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo'],
 'title': ['Đồ Bộ Nữ Đi Chơi Chất Thun Mát Size M - dưới 53kg / Size L Từ 54kg đến 65kg/ Size XL từ 65kg đến 72kg',
  'Bộ Pijama, Đồ Bộ Mặc Nhà Nữ Dáng Cộc xinh xắn',
  'Set Baggy Jeans Ống Suông Đai Nút Rách + Sơ mi rút dâY HOT',
  'Bộ mẫu hoạ tiết Sư Thích Minh Tuệ quần đùi tay ngắntừ 7KG ĐẾN 70KG , chất thun mềm - mịn - mát',
  '[3 SIZE 45-70KG) Đồ bộ LỤA ĐÙI CHẤM BI CHẤT VẢI LỤA HÀN MỀM MÁT thời trang',
  'Đồ bộ nữ mặc nhà dễ thương , bộ ngủ pijama ngắn hoạt hình

In [17]:
df = pd.DataFrame(dataProducts)

In [11]:
# Hàm cuộn từ từ xuống dưới cùng của trang cho đến khi get element được thì dừng
def getStarItem(driver, step=200, wait_time=0.1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    current_position = 0
    
    while current_position < last_height:
        driver.execute_script(f"window.scrollBy(0, {step});")
        sleep(wait_time)
        current_position += step
        elems_stars = driver.find_elements(By.CSS_SELECTOR, ".percent")
        if not (elems_stars == []):
            break
    if not (elems_stars == []):
        return [elem.text for elem in elems_stars]
    else :
        return ['N/A'] * 5

In [12]:
def getDetailItems(link):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(link)
    driver.implicitly_wait(2)
    # get original price
    try:
        elems_original = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[10]/div/div/div/span[1]")
        original = elems_original.text
    except NoSuchElementException:
        original = None
    # get ship price
    try:
        elems_ship_price = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[1]/div/div/div[3]/div/div[1]/div/div[1]/div[2]")
        ship_price = elems_ship_price.text
    except NoSuchElementException:
        ship_price = None
    # get return_
    try:
        elems_return = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[3]/div/div[2]/div[2]/div/div/div/div")
        return_ = elems_return.text
    except NoSuchElementException:
        return_ = None
    # get positive review
    try:
        elems_positive_review = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[1]/div[2]")
        positive_review = elems_positive_review.text
    except NoSuchElementException:
        positive_review = None
    # get ship on time
    try:
        elems_ship_on_time = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[2]/div[2]")
        ship_on_time = elems_ship_on_time.text
    except NoSuchElementException:
        ship_on_time = None
    # get rate response
    try:
        elems_rate_response = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[3]/div[2]")
        rate_response = elems_rate_response.text
    except NoSuchElementException:
        rate_response = None
    # get stars
    stars = getStarItem(driver)
    star_5 = stars[0]
    star_4 = stars[1]
    star_3 = stars[2]
    star_2 = stars[3]
    star_1 = stars[4]
    # into a record
    # into a record
    data = {
        "original": original,
        "ship_price": ship_price,
        "return": return_,
        "positive_review": positive_review,
        "ship_on_time": ship_on_time,
        "rate_response": rate_response,
        "one_star": star_1,
        "two_star": star_2,
        "three_star": star_3,
        "four_star": star_4,
        "five_star": star_5
    }
    driver.close()
    driver.quit()
    return data

In [13]:
detailProducts = {
    "original": [],
    "ship_price": [],
    "return": [],
    "positive_review": [],
    "ship_on_time": [],
    "rate_response": [],
    "one_star": [],
    "two_star": [],
    "three_star": [],
    "four_star": [],
    "five_star": []
}

In [21]:
# Sử dụng ThreadPoolExecutor để chạy đa luồng
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    # Sử dụng list comprehension để gọi hàm getDetailItems cho từng link trong df['links']
    results = list(tqdm(executor.map(getDetailItems, df['links']), total=len(df['links']), desc="Crawling product details"))







Crawling product details:   0%|          | 0/5 [02:06<?, ?it/s]


In [30]:
# Add additional columns
for key in results[0]:
    df[key] = [info[key] for info in results]

In [31]:
# save to csv
df.to_csv("lazada.csv", index = True)
df.head()

,type,title,links,price,discount_percent,sold,preview,location,original,ship_price,return,positive_review,ship_on_time,rate_response,one_star,two_star,three_star,four_star,five_star
0,quần áo,Đồ Bộ Nữ Đi Chơi Chất Thun Mát Size M - dưới 5...,https://www.lazada.vn/products/do-bo-nu-di-cho...,35.891 ₫,64% Off,3.0K Đã bán,(1016),Hồ Chí Minh,None,None,None,None,None,None,N/A,N/A,N/A,N/A,N/A
1,quần áo,"Bộ Pijama, Đồ Bộ Mặc Nhà Nữ Dáng Cộc xinh xắn",https://www.lazada.vn/products/bo-pijama-do-bo...,15.971 ₫,54% Off,229 Đã bán,(65),Hà Nội,None,None,None,None,None,None,N/A,N/A,N/A,N/A,N/A
2,quần áo,Set Baggy Jeans Ống Suông Đai Nút Rách + Sơ mi...,https://www.lazada.vn/products/set-baggy-jeans...,149.000 ₫,None,5 Đã bán,None,Hồ Chí Minh,None,None,None,None,None,None,N/A,N/A,N/A,N/A,N/A
3,quần áo,Bộ mẫu hoạ tiết Sư Thích Minh Tuệ quần đùi tay...,https://www.lazada.vn/products/bo-mau-hoa-tiet...,37.000 ₫,8% Off,78 Đã bán,(15),Cần Thơ,None,None,None,None,None,None,N/A,N/A,N/A,N/A,N/A
4,quần áo,[3 SIZE 45-70KG) Đồ bộ LỤA ĐÙI CHẤM BI CHẤT VẢ...,https://www.lazada.vn/products/3-size-45-70kg-...,33.454 ₫,66% Off,6.0K Đã bán,(1724),Hồ Chí Minh,None,None,None,None,None,None,20,17,28,44,1615
